## KI ersetzt die Simulation

Basierend auf den Winkeln der Motoren: $ x = (\phi_1, \phi_2) $ berechnet die Simulation die Position des TCP nach Verformung $y=(TCP_x, TCP_y)$.

Eine KI soll im Folgenden trainiert werden, die Simulation zu ersetzen und gleichfalls die TCP Position basierend auf den Motorwinkel vorherzusagen.

Mit diesem Skript kann ein Neuronales Netz trainiert werden, diese Vorhersagen zu treffen.

![sketch_TCP](img/sketch_TPC.png "Skizze Motorwinkel und TCP Verschiebung")

In [ ]:
#Importing data (DON'T CHANGE)
import os, numpy as np, random, pickle
X_train = np.load("data/X_train.npy")
Y_train = np.load("data/Y_train.npy")
_X_val  = np.load("data/X_val.npy"  )
_Y_val  = np.load("data/Y_val.npy"  )

import numpy as np
from sklearn.preprocessing import StandardScaler

# Instantiate and fit the scaler
scaler_X = StandardScaler()
scaler_X.fit(np.concatenate((X_train,_X_val)))
X_train = scaler_X.transform(X_train)
_X_val = scaler_X.transform(_X_val)
with open(f'data/scaler_X.pkl', 'wb') as f:
    pickle.dump(scaler_X, f)

In [ ]:
#Preparing data (DON'T CHANGE)
import torch
from X_999_functions_and_classes import MyDataset
dataset = MyDataset(X_train,Y_train)
# Convert data to PyTorch tensors
X_val = torch.tensor(_X_val, dtype=torch.float32)
Y_val = torch.tensor(_Y_val, dtype=torch.float32)
# Print dataset shapes
print(f'Training data shape: {X_train.shape}, {Y_train.shape}')
print(f'Validation data shape: {X_val.shape}, {Y_val.shape}')

In [ ]:
#Plot training data (DON'T CHANGE)
import matplotlib.pyplot as plt
import numpy as np

x_values, y_values = zip(*Y_train[:,-2:])
plt.scatter(x_values, y_values,marker=".",color=[[0,0.8,0]],s=0.1,label="Y_train")

x_range_all=(-0.5, 0.5)
y_range_all=(-0.5, 0)

plt.title(f"n_training_samples: {len(x_values)}")
plt.xlabel("X")
plt.ylabel("Y")
plt.xlim(-0.5, 0.5)
plt.ylim(-0.5, 0)
plt.gca().set_aspect('equal')
plt.legend()
plt.show()

In [ ]:
#Train the network
from X_999_functions_and_classes import train_nn

model,criterion,loss = train_nn(learning_rate   = 0.01,                 #change parameters here
                                batch_size      = 32,                   #change parameters here
                                layers          = [16,16],              #change parameters here
                                epochs          = 5,                    #change parameters here
                                dataset         = dataset,              
                                X_val           = X_val,
                                Y_val           = Y_val,
                                input_size      = len(X_val[0]),
                                output_size     = len(Y_val[0]),)
print(loss)

In [ ]:
#preprocess validation set (DON'T CHANGE)
# scale the X values for predicting
X_val_2_scaled = scaler_X.transform(X_val)
X_val_2_scaled = torch.tensor(X_val_2_scaled, dtype=torch.float32)
#do prediction
model.eval() # Set the model to evaluation mode
with torch.no_grad():
    # Forward pass for validation set
    Y_predicted = model(X_val)

In [ ]:
#plot overview validation set (DON'T CHANGE)
import matplotlib.pyplot as plt
fig = plt.figure(figsize=(8, 8), dpi=150)
ax = fig.add_subplot(111) 
ax.plot(Y_val[:,-2],Y_val[:,-1],color='green',linestyle='None',marker='.',markersize=1,linewidth=3,alpha=0.5,label="true path") 
Y_predicted_plot = np.array(Y_predicted[0])
ax.plot(Y_predicted[:,-2],Y_predicted[:,-1],color='red',linestyle='None',marker='.',markersize=1,linewidth=3,alpha=0.5,label="predicted path") 
limits=[-0.5,0.5,-0.5,0.0]
xlim_lower,xlim_upper,ylim_lower,ylim_upper = limits
ax.set_aspect('equal', adjustable='box')
ax.set_xlim([xlim_lower, xlim_upper])
#ax.set_xticks(np.arange(xlim_lower,xlim_upper+1,500))
ax.set_ylim([ylim_lower, ylim_upper])
#ax.set_yticks(np.arange(ylim_lower,ylim_upper+1,500))
ax.legend()
ax.set_title(f"")

In [ ]:
#plot detail validation set
import matplotlib.pyplot as plt

start_index = 10        #change parameters here
end_index = 20          #change parameters here

fig = plt.figure(figsize=(8, 8), dpi=150)
ax = fig.add_subplot(111) 
ax.plot(Y_val[start_index:end_index,0::2],Y_val[start_index:end_index,1::2],color='green',linestyle='None',marker='.',markersize=3,linewidth=3,alpha=0.5,label="true path") 
Y_predicted_plot = np.array(Y_predicted[0])
ax.plot(Y_predicted[start_index:end_index,0::2],Y_predicted[start_index:end_index,1::2],color='red',linestyle='None',marker='.',markersize=3,linewidth=3,alpha=0.5,label="predicted path") 
limits=[-0.5,0.5,-0.5,0.2]
xlim_lower,xlim_upper,ylim_lower,ylim_upper = limits
ax.set_aspect('equal', adjustable='box')
ax.set_xlim([xlim_lower, xlim_upper])
ax.set_ylim([ylim_lower, ylim_upper])
ax.legend()
ax.set_title(f"")